In [60]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display
from yahoo_finance import Share
import progressbar

# Read all stock symbols
The csv files that contain all the NASDAQ and NYSE symbols and other information are obtained from [NASDAQ website](http://www.nasdaq.com/screening/company-list.aspx).

In [13]:
# Get a list of stock symbols and their sectors
nasdaq_list = pd.read_csv('NASDAQ_LIST.csv', na_values=['n/a'])
nyse_list = pd.read_csv('NYSE_LIST.csv', na_values=['n/a'])
all_stocks = pd.concat([nasdaq_list, nyse_list])
all_stocks.drop('Unnamed: 8', axis=1, inplace=True)

# Drop stocks with unlabled sectors.
all_stocks.dropna(subset=['Sector'], inplace=True)

# Change row index
total_stocks = all_stocks.shape[0]
all_stocks.index = range(total_stocks)
display(all_stocks.head())

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
0,PIH,"1347 Property Insurance Holdings, Inc.",7.4260,$44.23M,2014.0,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih
1,TURN,180 Degree Capital Corp.,1.6399,$51.04M,NaN,Finance,Finance/Investors Services,http://www.nasdaq.com/symbol/turn
2,FLWS,"1-800 FLOWERS.COM, Inc.",10.3000,$672.95M,1999.0,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws
3,FCCY,1st Constitution Bancorp (NJ),17.1500,$137.67M,NaN,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy
4,SRCE,1st Source Corporation,45.5000,$1.18B,NaN,Finance,Major Banks,http://www.nasdaq.com/symbol/srce


# Retrieve financial information

Use a python library, yahoo-finance, to retrieve stock price and financial information from [Yahoo Finance website](https://finance.yahoo.com/). The information include:
1. **PE**: Price Earning Ratio
2. **PEG**: Price Earning Growth Ratio
3. **PS**: Price Sales Ratio
4. **PB**: Price Book Ratio
5. **SR**: Short Ratio
6. **DY**: Divident Yield (percentage)
7. **PCYH**: Percent Change From Year High
8. **PCYL**: Percent Change From Year Low

In [78]:
PE, PEG, PS, PB, SR, DY, PCYH, PCYL = [], [], [], [], [], [], [], []
bar = progressbar.ProgressBar()
for i in bar(range(total_stocks)):
    stock = Share(all_stocks['Symbol'][i])
    PE.append(stock.get_price_earnings_ratio())
    PEG.append(stock.get_price_earnings_growth_ratio())
    PS.append(stock.get_price_sales())
    PB.append(stock.get_price_book())
    SR.append(stock.get_short_ratio())
    DY.append(stock.get_dividend_yield())
    PCYH.append(stock.get_percent_change_from_year_high())
    PCYL.append(stock.get_percent_change_from_year_low())

100% (4966 of 4966) |#############################################################| Elapsed Time: 0:19:59 Time: 0:19:59


In [82]:
num_valid_stocks = len(PE)
print(num_valid_stocks)

4966


In [86]:
# Convert string data to float
def data_conversion(data):
    if data is None:
        return np.nan
    if '%' in data:
        return float(data.strip('%'))/100.0
    return float(data)

In [90]:
# Create the stock data frame
all_stock_df = pd.DataFrame({'PE':PE, 'PEG':PEG, 'PS':PS, 'PB':PB, 'SR':SR,
                             'DY':DY, 'PCYH':PCYH, 'PCYL':PCYL},
                            index = all_stocks['Symbol'][0:num_valid_stocks])
# Preprocess the data
all_stock_df = all_stock_df.apply(np.vectorize(data_conversion))
all_stock_df['DY'].fillna(0.0, inplace=True) # some stocks don't pay divident

# Add Sectors
all_stock_df['SECTOR'] = pd.Series(all_stocks['Sector'][0:num_valid_stocks].as_matrix(), 
                                index=all_stock_df.index)

all_stock_df.dropna(thresh=all_stock_df.shape[1], inplace=True)
display(all_stock_df.head())

,DY,PB,PCYH,PCYL,PE,PEG,PS,SR,SECTOR
Symbol,,,,,,,,,
PIH,0.00,0.951,-0.171156,0.320283,33.1538,0.00,1.3706,0.52,Finance
FLWS,0.00,2.460,-0.105300,0.297700,27.3500,1.13,0.5700,6.33,Consumer Services
FCCY,1.16,1.300,-0.172700,0.474400,15.7000,0.86,3.1800,0.64,Finance
SRCE,1.63,1.760,-0.087200,0.522200,20.0600,1.86,4.7100,3.36,Finance
JOBS,0.00,3.410,-0.028300,0.555700,27.4800,7.42,7.5400,3.50,Technology


In [91]:
display(all_stock_df.describe())

,DY,PB,PCYH,PCYL,PE,PEG,PS,SR
count,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000
mean,1.923789,5.262124,-0.137261,1.036934,50.919360,4.647964,3.663936,5.036375
std,2.590263,32.518166,0.131984,33.707553,219.507561,140.381934,14.516095,5.342447
min,0.000000,0.070700,-0.942500,0.000000,0.075000,-4098.680000,0.010000,0.000000
25%,0.000000,1.360000,-0.188500,0.173100,15.500000,0.000000,1.010000,1.830000
50%,1.250000,2.130000,-0.103400,0.315600,21.670000,1.300000,2.250000,3.470000
75%,2.720000,3.760000,-0.041500,0.518400,34.030000,2.290000,4.192800,6.510000
max,35.740000,1191.530000,0.295000,1805.749800,6385.000000,4299.500000,529.420000,58.350000


In [85]:
# Saving data to disk
all_stock_df.to_csv('all_stock_data.csv')